In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# -----------------------------
# CONFIG
# -----------------------------
DATASET_DIR = "/kaggle/input/chest-xray-pneumonia/chest_xray"
IMG_SIZE = 128  # Reduced size for Raspberry Pi compatibility
BATCH_SIZE = 16
EPOCHS = 15
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -----------------------------
# DATA PREPARATION
# -----------------------------
def collect_image_paths(dataset_root):
    import glob
    train_path = os.path.join(dataset_root, "train")
    test_path  = os.path.join(dataset_root, "test")

    def make_df(path):
        normal = glob.glob(os.path.join(path, "NORMAL", "*"))
        pneumonia = glob.glob(os.path.join(path, "PNEUMONIA", "*"))
        return pd.DataFrame({
            "image": normal + pneumonia,
            "class": ["Normal"] * len(normal) + ["Pneumonia"] * len(pneumonia)
        })

    return make_df(train_path), make_df(test_path)

def build_generator(df, img_size, batch_size, shuffle=True, augment=False):
    datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2 if augment else 0.0,
        width_shift_range=0.15 if augment else 0.0,
        height_shift_range=0.15 if augment else 0.0,
        horizontal_flip=augment,
        rotation_range=10 if augment else 0.0,
        brightness_range=[0.8, 1.2] if augment else None
    )
    return datagen.flow_from_dataframe(
        df, x_col="image", y_col="class",
        target_size=(img_size, img_size),
        class_mode="binary",
        batch_size=batch_size, shuffle=shuffle
    )



2025-11-20 09:22:31.005602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763630551.326516      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763630551.410799      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# -----------------------------
# LOAD DATA
# -----------------------------
df_train_all, df_test = collect_image_paths(DATASET_DIR)
train_df, val_df = train_test_split(df_train_all, test_size=0.2,
                                    stratify=df_train_all["class"], random_state=SEED)

train_gen = build_generator(train_df, IMG_SIZE, BATCH_SIZE, augment=True)
val_gen   = build_generator(val_df, IMG_SIZE, BATCH_SIZE)
test_gen  = build_generator(df_test, IMG_SIZE, 1, shuffle=False)



Found 4172 validated image filenames belonging to 2 classes.
Found 1044 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [3]:
# -----------------------------
# MODEL (Optimized VGG16 for Raspberry Pi)
# -----------------------------

def create_lightweight_vgg16(img_size=128):
    """Create optimized VGG16-based model for Raspberry Pi with high accuracy"""
    
    # Load pre-trained VGG16 with smaller input size
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(img_size, img_size, 3)
    )
    
    # Freeze early layers, fine-tune later layers
    for layer in base_model.layers[:10]:  # Freeze first 10 layers
        layer.trainable = False
    for layer in base_model.layers[10:]:  # Fine-tune later layers
        layer.trainable = True
    
    # Custom classifier head optimized for medical images
    inputs = keras.Input(shape=(img_size, img_size, 3))
    x = base_model(inputs, training=False)
    
    # Global Average Pooling instead of Flatten to reduce parameters
    x = layers.GlobalAveragePooling2D()(x)
    
    # Attention mechanism for better feature selection
    attention = layers.Dense(x.shape[-1], activation='sigmoid')(x)
    x = layers.Multiply()([x, attention])
    
    # Enhanced classifier with batch normalization and careful regularization
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    return model

# Create the optimized VGG16 model
model = create_lightweight_vgg16(img_size=IMG_SIZE)

# Use lower learning rate for fine-tuning
optimizer = keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy', keras.metrics.AUC(name='auc')]
)

model.summary()

print(f"✅ Optimized VGG16 created with {model.count_params():,} parameters")
print("🚀 Optimized for Raspberry Pi 3 with high accuracy!")


2025-11-20 09:23:15.659700: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 4, 4, 512) │ 14,714,688 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    262,656 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 512)       │          0 │ global_average_p… │
│                     │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_3[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,151,681 (57.80 MB)

 Trainable params: 13,415,297 (51.18 MB)

 Non-trainable params: 1,736,384 (6.62 MB)

✅ Optimized VGG16 created with 15,151,681 parameters
🚀 Optimized for Raspberry Pi 3 with high accuracy!


In [4]:

# -----------------------------
# TRAINING WITH ENHANCED CALLBACKS
# -----------------------------
# Enhanced callbacks for better training
es = callbacks.EarlyStopping(
    monitor="val_auc", 
    patience=8, 
    restore_best_weights=True,
    mode='max',
    verbose=1
)

rlrop = callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    mode='max',
    verbose=1
)

checkpoint = callbacks.ModelCheckpoint(
    "best_vgg16_pneumonia.h5",
    monitor="val_auc",
    save_best_only=True,
    mode='max',
    verbose=1
)

# Class weights for imbalanced data (usually more pneumonia cases)
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"📊 Class weights: {class_weight_dict}")

print("🎯 Starting training with optimized VGG16...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[es, rlrop, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)



📊 Class weights: {0: 1.9440820130475303, 1: 0.6731203614069055}
🎯 Starting training with optimized VGG16...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.7109 - binary_accuracy: 0.6261 - loss: 0.6841
Epoch 1: val_auc improved from -inf to 0.97029, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━ 766s 3s/step - auc: 0.7112 - binary_accuracy: 0.6264 - loss: 0.6836 - val_auc: 0.9703 - val_binary_accuracy: 0.3649 - val_loss: 0.7336 - learning_rate: 1.0000e-05
Epoch 2/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9062 - binary_accuracy: 0.7673 - loss: 0.4274
Epoch 2: val_auc improved from 0.97029 to 0.98289, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━ 752s 3s/step - auc: 0.9063 - binary_accuracy: 0.7673 - loss: 0.4273 - val_auc: 0.9829 - val_binary_accuracy: 0.6370 - val_loss: 0.7959 - learning_rate: 1.0000e-05
Epoch 3/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9415 - binary_accuracy: 0.8039 - loss: 0.3639
Epoch 3: val_auc improved from 0.98289 to 0.98926, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━

In [ ]:

# -----------------------------
# TRAINING WITH ENHANCED CALLBACKS
# -----------------------------
# Enhanced callbacks for better training
es = callbacks.EarlyStopping(
    monitor="val_auc", 
    patience=8, 
    restore_best_weights=True,
    mode='max',
    verbose=1
)

rlrop = callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    mode='max',
    verbose=1
)

checkpoint = callbacks.ModelCheckpoint(
    "best_vgg16_pneumonia.h5",
    monitor="val_auc",
    save_best_only=True,
    mode='max',
    verbose=1
)

# Class weights for imbalanced data (usually more pneumonia cases)
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"📊 Class weights: {class_weight_dict}")

print("🎯 Starting training with optimized VGG16...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[es, rlrop, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)



📊 Class weights: {0: 1.9440820130475303, 1: 0.6731203614069055}
🎯 Starting training with optimized VGG16...
Epoch 1/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.7963 - binary_accuracy: 0.6695 - loss: 0.5922

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_auc improved from -inf to 0.96964, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━ 939s 4s/step - auc: 0.7965 - binary_accuracy: 0.6697 - loss: 0.5919 - val_auc: 0.9696 - val_binary_accuracy: 0.4205 - val_loss: 0.6989 - learning_rate: 1.0000e-05
Epoch 2/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9204 - binary_accuracy: 0.7872 - loss: 0.4065
Epoch 2: val_auc improved from 0.96964 to 0.98244, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━ 880s 3s/step - auc: 0.9205 - binary_accuracy: 0.7873 - loss: 0.4064 - val_auc: 0.9824 - val_binary_accuracy: 0.7998 - val_loss: 0.6002 - learning_rate: 1.0000e-05
Epoch 3/15
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - auc: 0.9471 - binary_accuracy: 0.8321 - loss: 0.3397
Epoch 3: val_auc improved from 0.98244 to 0.98381, saving model to best_vgg16_pneumonia.h5
261/261 ━━━━━━━━━━━━━━━━━━━━ 882s 3s/step - auc: 0.9471 - binary_accuracy: 0.8321 - loss: 0.3396 - val_auc: 0.9838 - val_binary_accur

In [5]:
# Load best model
model = keras.models.load_model("best_vgg16_pneumonia.h5")
print("✅ Best model loaded for evaluation")


✅ Best model loaded for evaluation


In [1]:
import matplotlib.pyplot as plt

# ---- Accuracy Curve ----
plt.figure(figsize=(6,5))
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Train Accuracy", "Validation Accuracy"])
plt.title("Training vs Validation Accuracy")
plt.show()

# ---- Loss Curve ----
plt.figure(figsize=(6,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train Loss", "Validation Loss"])
plt.title("Training vs Validation Loss")
plt.show()


NameError: name 'history' is not defined

<Figure size 600x500 with 0 Axes>

In [6]:

# -----------------------------
# EVALUATION
# -----------------------------
print("🧪 Evaluating model on test set...")
preds = model.predict(test_gen, verbose=1)
pred_labels = (preds > 0.5).astype(int).reshape(-1)
y_true = df_test["class"].map({"Normal":0, "Pneumonia":1}).values

test_accuracy = accuracy_score(y_true, pred_labels)
test_auc = roc_auc_score(y_true, preds)

print(f"🎯 Test Accuracy: {test_accuracy:.4f}")
print(f"📊 Test ROC-AUC: {test_auc:.4f}")
print(classification_report(y_true, pred_labels, target_names=["Normal","Pneumonia"]))



🧪 Evaluating model on test set...
624/624 ━━━━━━━━━━━━━━━━━━━━ 55s 87ms/step
🎯 Test Accuracy: 0.9279
📊 Test ROC-AUC: 0.9705
              precision    recall  f1-score   support

      Normal       0.96      0.84      0.90       234
   Pneumonia       0.91      0.98      0.94       390

    accuracy                           0.93       624
   macro avg       0.94      0.91      0.92       624
weighted avg       0.93      0.93      0.93       624



In [7]:
# -----------------------------
# TFLITE QUANTIZATION FOR RASPBERRY PI 3
# -----------------------------
TFLITE_MODEL_PATH = "pneumonia_vgg16_quantized.tflite"
TFLITE_FLOAT_MODEL_PATH = "pneumonia_vgg16_float.tflite"

def representative_dataset_gen():
    """Representative dataset for quantization"""
    for _ in range(100):
        # Generate random data that matches training distribution
        data = np.random.uniform(0, 1, (1, IMG_SIZE, IMG_SIZE, 3)).astype(np.float32)
        yield [data]

print("🔄 Converting to TFLite for Raspberry Pi 3...")

# Strategy 1: First create a float32 model (most compatible)
converter_float = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter_float.convert()

with open(TFLITE_FLOAT_MODEL_PATH, "wb") as f:
    f.write(tflite_float_model)
print(f"✅ Float32 TFLite model saved: {TFLITE_FLOAT_MODEL_PATH}")

# Strategy 2: Try dynamic range quantization (good balance)
try:
    converter_dynamic = tf.lite.TFLiteConverter.from_keras_model(model)
    converter_dynamic.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_dynamic_model = converter_dynamic.convert()
    
    dynamic_path = "pneumonia_vgg16_dynamic_quant.tflite"
    with open(dynamic_path, "wb") as f:
        f.write(tflite_dynamic_model)
    print(f"✅ Dynamic range quantized model saved: {dynamic_path}")
except Exception as e:
    print(f"⚠️ Dynamic quantization failed: {e}")

# Strategy 3: Try full INT8 quantization (smallest size)
try:
    converter_int8 = tf.lite.TFLiteConverter.from_keras_model(model)
    converter_int8.optimizations = [tf.lite.Optimize.DEFAULT]
    converter_int8.representative_dataset = representative_dataset_gen
    
    # For maximum compatibility, don't force input/output types
    # converter_int8.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter_int8.inference_input_type = tf.uint8
    # converter_int8.inference_output_type = tf.uint8
    
    tflite_int8_model = converter_int8.convert()
    
    with open(TFLITE_MODEL_PATH, "wb") as f:
        f.write(tflite_int8_model)
    print(f"✅ INT8 quantized model saved: {TFLITE_MODEL_PATH}")
except Exception as e:
    print(f"⚠️ INT8 quantization failed: {e}")
    # Fallback to dynamic quantization
    TFLITE_MODEL_PATH = dynamic_path

# Test TFLite model
print("🧪 Testing TFLite model...")
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
print("✅ TFLite model loaded successfully!")

# Model size analysis
print("\n📊 MODEL SIZE ANALYSIS FOR RASPBERRY PI 3:")
original_size = os.path.getsize("best_vgg16_pneumonia.h5") / (1024 * 1024)
tflite_size = os.path.getsize(TFLITE_MODEL_PATH) / (1024 * 1024)

print(f"   Original H5: {original_size:.2f} MB")
print(f"   TFLite Quantized: {tflite_size:.2f} MB")
print(f"   Compression: {original_size/tflite_size:.1f}x smaller")

# Memory requirements estimation
print(f"   Estimated RAM usage: {tflite_size * 2:.1f} MB (2x model size)")

# Performance tips for Raspberry Pi
print("\n🎯 RASPBERRY PI 3 DEPLOYMENT TIPS:")
print("   1. Use float32 model for best accuracy")
print("   2. Use quantized model for faster inference")
print("   3. Expected inference time: 200-500ms per image")
print("   4. Recommended: Use with TensorFlow Lite Runtime")
print("   5. Enable ARM NEON optimizations for better performance")

# Save training history for analysis
history_df = pd.DataFrame(history.history)
history_df.to_csv("vgg16_training_history.csv", index=False)
print("✅ Training history saved!")

🔄 Converting to TFLite for Raspberry Pi 3...
Saved artifact at '/tmp/tmp388torbp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131940930129360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930130896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930127440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930126672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920152336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920153104: TensorSpec(shape=(), 

W0000 00:00:1760387529.132182      37 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1760387529.132326      37 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1760387529.160897      37 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


✅ Float32 TFLite model saved: pneumonia_vgg16_float.tflite
Saved artifact at '/tmp/tmptzf9823b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131940930129360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930130896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930127440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930126672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920152336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920153104: TensorS

W0000 00:00:1760387537.511725      37 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1760387537.511765      37 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


✅ Dynamic range quantized model saved: pneumonia_vgg16_dynamic_quant.tflite
Saved artifact at '/tmp/tmpy2ufsn01'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  131940930129360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930130896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930127440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930126672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131940930131856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920152336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131939920151760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1319399

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1760387546.162704      37 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1760387546.162742      37 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


✅ INT8 quantized model saved: pneumonia_vgg16_quantized.tflite
🧪 Testing TFLite model...
✅ TFLite model loaded successfully!

📊 MODEL SIZE ANALYSIS FOR RASPBERRY PI 3:
   Original H5: 160.28 MB
   TFLite Quantized: 14.59 MB
   Compression: 11.0x smaller
   Estimated RAM usage: 29.2 MB (2x model size)

🎯 RASPBERRY PI 3 DEPLOYMENT TIPS:
   1. Use float32 model for best accuracy
   2. Use quantized model for faster inference
   3. Expected inference time: 200-500ms per image
   4. Recommended: Use with TensorFlow Lite Runtime
   5. Enable ARM NEON optimizations for better performance
✅ Training history saved!


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
from tensorflow.keras.models import load_model

# Load your H5 model (no compile needed)
model = load_model("/kaggle/input/majorproject/best_vgg16_pneumonia.h5", compile=False)

# Save as SavedModel (folder path, no save_format)
model.save("best_vgg16_saved_model")  # This will create a folder


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=best_vgg16_saved_model.